In [1]:
import numpy as np
import pandas as pd
from data_reader import read_consumption_and_weather
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
import math

torch.manual_seed(1)

In [2]:
_, _, df = read_consumption_and_weather()

df1 = df['NO1']
df2 = df['NO2']
df3 = df['NO3']
df4 = df['NO4']
df5 = df['NO5']

Holes by length and occurrences in column NO3:
holes
1        1
dtype: int64
Filling holes up to length 3
Any remaining holes after interpolation? False


In [3]:
df = df3[['consumption']]

df

,consumption
2017-05-01 00:00:00+00:00,2674.838000
2017-05-01 01:00:00+00:00,2678.188000
2017-05-01 02:00:00+00:00,2678.341000
2017-05-01 03:00:00+00:00,2684.172000
2017-05-01 04:00:00+00:00,2733.345000
...,...
2023-01-21 19:00:00+00:00,3702.984824
2023-01-21 20:00:00+00:00,3613.726395
2023-01-21 21:00:00+00:00,3547.775302
2023-01-21 22:00:00+00:00,3483.455353


In [4]:
# Get the 0.8 first timesteps for training and 0.2 last one for testing
df_train, df_test = train_test_split(df, test_size=0.1, shuffle=False)
# Divide into train, validation
df_train, df_validation = train_test_split(df_train, test_size=0.1, shuffle=False)

In [5]:
# Save for undoing normalization in testing
train_mean = df_train.mean()
train_std = df_train.std()

# Normalize train and test set
df_train = (df_train - train_mean) / train_std
df_validation = (df_validation - train_mean) / train_std
df_test = (df_test - train_mean) / train_std

In [6]:
df_train

,consumption
2017-05-01 00:00:00+00:00,-0.800239
2017-05-01 01:00:00+00:00,-0.793282
2017-05-01 02:00:00+00:00,-0.792964
2017-05-01 03:00:00+00:00,-0.780855
2017-05-01 04:00:00+00:00,-0.678736
...,...
2021-12-20 07:00:00+00:00,1.481909
2021-12-20 08:00:00+00:00,1.561384
2021-12-20 09:00:00+00:00,1.565933
2021-12-20 10:00:00+00:00,1.516567


In [ ]:
# For use in the transformer model
class PositionalEncoder(nn.Module):

    def __init__(self, dropout=0.1, max_seq_len=5000, d_model=512, batch_first=False):
        super().__init__()

        self.d_model = d_model
        self.dropout = dropout
        self.batch_first = batch_first
        self.x_dim = 1 if batch_first else 0

        position = torch.arange(max_seq_len).unsqueeze(1)

        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))

        pe = torch.zeros(max_seq_len, 1, d_model)

        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)

    
    def forward(self, x):
        x = x + self.pe[:x.size(self.x_dim)]
        return self.dropout(x)
    

In [ ]:

class TransformerModel(nn.Module):

    def __init__(self):

        input_size = 24
        output_size = 1
        dim=512

        # ENCODER
        self.encoder_input_layer = nn.Linear(
            in_features=input_size,
            out_features=dim
        )

        self.positional_encoding_layer = PositionalEncoder(
            d_model=dim,
            dropout=0.1,
            max_seq_len=5000
        )

        n_heads = 8

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim,
            nhead=n_heads,
            batch_first=True
        )

        self.encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layer,
            num_layers=4,
            norm=None
        )

        # DECODER
        self.decoder_input_layer = nn.Linear(
            in_features=input_size,
            out_features=dim
        )

        decoder_layer = nn.TransformerDecoderLayer(
            d_model=dim,
            nhead=n_heads,
            batch_first=True
        )

        self.decoder = nn.TransformerDecoder(
            decoder_layer=decoder_layer,
            num_layers=4,
            norm=None
        )

        self.linear_mapping = nn.Linear(
            in_features=dim,
            out_features=output_size
        )


    def forward():
        return


